# Car Price Prediction Service - Feature Pipeline

## Installing Hopsworks, the feature store used in this project.


In [ ]:
! pip install -U hopsworks --quiet

Run in either "Backfill" or "Normal" operation. 
Backfill is for historical data (here, data scraped from cardekho.com,  imported from kaggle. Traning will be done with this dataset.
Normal operation is for batch inference. Simulating, real data streamed from any storage store. 

IF BACKFILL==True, we will load our DataFrame with data from the car_prices_clean_v2.csv file

ELSE BACKFILL==False, we will load our DataFrame with one (synthetic) car detail sample

In [38]:
import random
import pandas as pd
import hopsworks

BACKFILL = False

## Synthetic Data generation

In [34]:
def generate_vehicle(full_name, selling_price_max, selling_price_min, age_max, age_min,
                     km_driven_max, km_driven_min, mileage_max, mileage_min, engine_max,
                     engine_min, max_power_max, max_power_min, seats, seller_type, 
                     owner_type, fuel_type, transmission_type):
  """
    Returns a single vehicle as a single row in a DataFrame
  """
  df = pd.DataFrame({  "selling_price": [random.uniform(selling_price_max, selling_price_min)],
                       "age": [random.randint(age_min, age_max)],
                       "km_driven": [random.randint(km_driven_min, km_driven_max)],
                       "mileage": [random.uniform(mileage_max, mileage_min)],
                       "engine": [random.uniform(engine_max, engine_min)],
                       "max_power": [random.uniform(max_power_max, max_power_min)]
                      })
  df['full_name'] = full_name
  df['seller_type'] = seller_type
  df['owner_type'] = owner_type
  df['fuel_type'] = fuel_type
  df['transmission_type'] = transmission_type
  df['seats'] = seats

  df = df[['full_name', 'selling_price', 'seller_type', 'km_driven', 'owner_type', 'fuel_type',
           'transmission_type', 'mileage', 'engine', 'max_power', 'seats', 'age']]
  return df

def get_random_vehicle():
  """
    Returns a DataFrame containing one random vehicle chosen from the synthetic data
  """
  Maruti_Swift = generate_vehicle('Maruti Swift Dzire VDI', 6, 1, 10, 3,
                     100000, 100000, 22, 16, 800,
                     600, 70, 30, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  Hyundai_i20_Asta = generate_vehicle('Hyundai_i20_Asta', 6, 3, 8, 3,
                     100000, 20000, 18, 12, 1400,
                     1200, 90, 70, 5, 'Individual', 'First Owner', 'Petrol', 'Manual')
  Ford_Ecosport = generate_vehicle('Ford_Ecosport', 7, 4, 8, 4,
                     100000, 20000, 18, 12, 1700,
                     1200, 120, 80, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  Toyota_Yaris = generate_vehicle('Toyota_Yaris', 7, 4, 10, 5,
                     100000, 20000, 18, 12, 1700,
                     1200, 120, 80, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  Volkswagen_Vento = generate_vehicle('Volkswagen_Vento', 7, 4, 9, 5,
                     100000, 20000, 18, 12, 1700,
                     1200, 120, 80, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  Honda_City = generate_vehicle('Honda_City', 7, 4, 7, 5,
                     100000, 20000, 18, 12, 1700,
                     1200, 120, 80, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  # Nissan_Micra = generate_vehicle(full_name, selling_price_max, selling_price_min, year_max, year_min,
  #                    km_driven_max, km_driven_min, mileage_max, mileage_min, engine_max,
  #                    engine_min, max_power_max, max_power_min, seats_max)
  # Renault_Duster = generate_vehicle(full_name, selling_price_max, selling_price_min, year_max, year_min,
  #                    km_driven_max, km_driven_min, mileage_max, mileage_min, engine_max,
  #                    engine_min, max_power_max, max_power_min, seats_max)
  Tata_Indigo = generate_vehicle('Tata_Indigo', 6, 3, 2016, 2010,
                     100000, 20000, 18, 12, 1400,
                     1200, 90, 70, 5, 'Dealer', 'First Owner', 'Diesel', 'Manual')
  # Mercedes_Benz = generate_vehicle(full_name, selling_price_max, selling_price_min, year_max, year_min,
  #                    km_driven_max, km_driven_min, mileage_max, mileage_min, engine_max,
  #                    engine_min, max_power_max, max_power_min, seats_max)
  pick_random = random.uniform(0,7)
  if pick_random >= 6:
      car_df = Maruti_Swift
  elif pick_random >= 5:
      car_df = Tata_Indigo
  elif pick_random >= 4:
      car_df = Honda_City
  elif pick_random >= 3:
      car_df = Volkswagen_Vento
  elif pick_random >= 2:
      car_df = Toyota_Yaris
  elif pick_random >= 1:
      car_df = Hyundai_i20_Asta
  else:
      car_df = Ford_Ecosport

  return car_df

In [35]:
if BACKFILL == True:
    car_df = pd.read_csv("/content/car_prices_clean_v2.csv")
else:
    car_df = get_random_vehicle()
    
car_df.head()

,full_name,selling_price,seller_type,km_driven,owner_type,fuel_type,transmission_type,mileage,engine,max_power,seats,age
0,Toyota Land Cruiser 200 V8 Petrol,92.0,Dealer,80000,First Owner,Petrol,Automatic,9.00,4461.0,285.40,8.0,6
1,Land Rover Range Rover Sport 3.0 D SE,92.0,Dealer,9500,First Owner,Diesel,Automatic,12.65,2993.0,296.00,5.0,5
2,Toyota Land Cruiser Prado VXL,87.5,Dealer,36000,First Owner,Diesel,Automatic,11.00,2982.0,170.63,7.0,3
3,BMW 7 Series 730Ld DPE Signature,85.0,Individual,50000,First Owner,Diesel,Automatic,16.77,2993.0,261.49,5.0,4
4,BMW 7 Series 730Ld DPE Signature,85.0,Dealer,19000,First Owner,Diesel,Automatic,17.66,2993.0,355.37,4.0,4


In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

In [37]:
car_fg = fs.get_or_create_feature_group(name="car_prices",
                                  version=1,
                                  primary_key=["age", "km_driven",	"mileage",	"engine",	"max_power",	"full_name",	"seller_type",	"owner_type",	"fuel_type"],
                                  description="Car Prices Dataset"
                                 )
car_fg.insert(car_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3342/fs/3289/fg/5554


Uploading Dataframe: 0.00% |          | Rows 0/10321 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/car_prices_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f951faea450>, None)